# **Probability and Statistics (Week 1)**

```
.------------------------------------.
|   __  ____  ______  _  ___ _____   |
|  |  \/  \ \/ / __ )/ |/ _ \___  |  |
|  | |\/| |\  /|  _ \| | | | | / /   |
|  | |  | |/  \| |_) | | |_| |/ /    |
|  |_|  |_/_/\_\____/|_|\___//_/     |
'------------------------------------'

```


## **Pre-configurate the Notebook**

### **Switching to the R Kernel on Colab**

By default, Google Colab uses Python as its programming language. To use R instead, you’ll need to manually switch the kernel by going to **Runtime > Change runtime type**, and selecting R as the kernel. This allows you to run R code in the Colab environment.

However, our notebook is already configured to use R by default. Unless something goes wrong, you shouldn’t need to manually change runtime type.

### **Import Required Datasets and Packages**
**Run the following lines of code**:

In [9]:
#Do not modify

setwd("/content")

# Remove `MXB107-Notebooks` if exists,
if (dir.exists("MXB107-Notebooks")) {
  system("rm -rf MXB107-Notebooks")
}

# Fork the repository
system("git clone https://github.com/edelweiss611428/MXB107-Notebooks.git")

# Change working directory to "MXB107-Notebooks"
setwd("MXB107-Notebooks")

#
invisible(source("R/preConfigurated.R"))

**Do not modify the following**

In [11]:
if (!require("testthat")) install.packages("testthat"); library("testthat")

test_that("Test if all packages have been loaded", {

  expect_true(all(c("ggplot2", "dplyr", "stringr", "magrittr") %in% loadedNamespaces()))

})

test_that("Test if all utility functions have been loaded", {
  expect_true(exists("skewness"))
  expect_true(exists("kurtosis"))
})

Test passed 🌈
Test passed 😀


## **Data Wrangling in R via `dplyr` and `magrittr`**

Data wrangling is the process of cleaning, transforming, and reshaping data for analysis. The `dplyr` package, often used with the pipe operator `%>%` from `magrittr`, provides a set of intuitive functions for these tasks.

### **Can We Do This via Base R?**

Yes, but the syntax tends to be more verbose and less readable. For demonstration, we will load the CSV file `MXB107_2025_ClassInfo.csv`.

In [2]:
MXB107_Info = read.csv("./datasets/MXB107_2025_ClassInfo.csv")
str(MXB107_Info)

'data.frame':	12 obs. of  8 variables:
 $ Class         : chr  "LEC01 01" "LEC01 01" "PRC01 01" "PRC01 01" ...
 $ Type          : chr  "Lecture (Internal)" "Lecture (Online)" "Practical (Online)" "Practical (Internal)" ...
 $ Day           : chr  "Wed" "Wed" "Wed" "Thu" ...
 $ Location      : chr  "GP B117" "Online" "Online" "GP D413" ...
 $ Limit         : int  240 1000 30 35 30 35 25 30 35 35 ...
 $ Teaching_Staff: chr  "Chris Drovandi" "Chris Drovandi" "Narayan Srinivasan" "Narayan Srinivasan" ...
 $ From          : int  11 11 16 16 16 9 14 9 9 11 ...
 $ To            : int  13 13 18 18 18 11 16 11 11 13 ...


We want to extract all classes that start after 9:00 AM on either Thursday or Friday and are not online.

In [3]:
notOnline = MXB107_Info$Location != "Online"
onThursday = MXB107_Info$Day %in% c("Thu", "Fri")
startAfter9AM = MXB107_Info$From > 9
subset(MXB107_Info, notOnline & onThursday & startAfter9AM)

,Class,Type,Day,Location,Limit,Teaching_Staff,From,To
,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
4,PRC01 01,Practical (Internal),Thu,GP D413,35,Narayan Srinivasan,16,18
7,PRC01 07,Practical (Internal),Thu,GP S520,25,Ryan Kelly,14,16
10,PRC01 04,Practical (Internal),Fri,GP G216,35,Arwen Nugteren,11,13
11,PRC01 05,Practical (Internal),Fri,GP S502,35,Arwen Nugteren,15,17
12,PRC01 06,Practical (Internal),Fri,GP S519,35,Minh Long Nguyen,15,17


The `dplyr` and `magrittr` packages simplify and streamline data manipulation by providing a data-processing pipeline.

When we write

```
input %>% do_something_1() %>% do_something_2()
```

The pipe operator %>% takes the output from the expression on the left and passes it as the **first argument** to the function on the right. This allows chaining multiple operations in a clear, readable sequence.


In base R, it would be something like

```
output1 = do_something_1(input)
output2 = do_something_2(output1)
```
Which one looks better to you?




Another great feature of `dplyr` is that it natively understands data frame columns by internally converting data frames into **tibbles** — a more user-friendly data frame format.

This means you don’t need to repeatedly write `MXB107_Info$From` or similar; inside `dplyr` **verbs** like `filter()`, you can simply refer to columns by their names, such as `From` and `To`, which makes the code much cleaner and easier to read.

In [4]:
MXB107_Info %>%
  filter(Location != "Online",
         Day %in% c("Thu", "Fri"),
         From > 9) -> results
print(results)

     Class                 Type Day Location Limit     Teaching_Staff From To
1 PRC01 01 Practical (Internal) Thu  GP D413    35 Narayan Srinivasan   16 18
2 PRC01 07 Practical (Internal) Thu  GP S520    25         Ryan Kelly   14 16
3 PRC01 04 Practical (Internal) Fri  GP G216    35     Arwen Nugteren   11 13
4 PRC01 05 Practical (Internal) Fri  GP S502    35     Arwen Nugteren   15 17
5 PRC01 06 Practical (Internal) Fri  GP S519    35   Minh Long Nguyen   15 17


That indeed looks much better! Here the right-arrow assignment operator really shines.

### **Common `dplyr` Verbs**

#### **filter() — subset rows by condition**

In [7]:
MXB107_Info %>%
  filter(Type == "Lecture (Internal)")

MXB107_Info %>%
  filter(str_detect(Type, "Practical"), #base R: str_detect(MXB107_Info$Type, pattern = "Practical")
         Day == "Wed")

Class,Type,Day,Location,Limit,Teaching_Staff,From,To
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
LEC01 01,Lecture (Internal),Wed,GP B117,240,Chris Drovandi,11,13


Class,Type,Day,Location,Limit,Teaching_Staff,From,To
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
PRC01 01,Practical (Online),Wed,Online,30,Narayan Srinivasan,16,18
